# Query Understanding: Intents + Slot Mapping (Clean Notebook)

This is a compact notebook to:
- load a trained **intent classifier** (ML.NET) or use a minimal rule-based fallback,
- perform **slot mapping** using a **domain dictionary** (departments, roles, locations) and
- extract dates/numbers via **Microsoft.Recognizers.Text**.

Keep your existing, more verbose notebook as the study guide. Use this one for daily work.

**NuGet (project):**
- `Microsoft.ML`
- `Microsoft.Recognizers.Text`
- `Microsoft.Recognizers.Text.DateTime`
- `Microsoft.Recognizers.Text.Number`
- `FuzzySharp`
- `System.Text.Json`

> This notebook assumes those are referenced in your project. In .NET Interactive, use `#r "nuget: ..."` if needed.

In [1]:
#r "nuget: Microsoft.ML, 4.0.2"
#r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"
#r "nuget: Microsoft.Recognizers.Text.Number, 1.8.13"
#r "nuget: FuzzySharp, 2.0.2"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Text.RegularExpressions;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;
using Microsoft.Recognizers.Text.Number;
using FuzzySharp;

Installed Packages FuzzySharp, 2.0.2 Microsoft.ML, 4.0.2 Microsoft.Recognizers.Text.DateTime, 1.8.13 Microsoft.Recognizers.Text.Number, 1.8.13

In [2]:
public enum Intent
{
    GET_CONTACT_ADDRESS,
    GET_CONTACT_INFO,
    GET_CONTACT_PHONE,
    GET_CONTACT_EMAIL,
    FILTER_BY_HIRE_DATE,
    FILTER_BY_DEPARTMENT,
    FILTER_BY_LOCATION,
    FILTER_BY_BIRTHDAY,
    UNKNOWN
}

public sealed class QuerySlots
{
    public string? Field { get; set; }                // e.g., "OriginalHireDate"
    public string? Operator { get; set; }             // e.g., "before", "after", "between"
    public DateTime? DateStart { get; set; }
    public DateTime? DateEnd { get; set; }
    public string? Department { get; set; }           // canonical department
    public string? Role { get; set; }                 // canonical role
    public string? Location { get; set; }             // canonical location
    public List<string> Names { get; set; } = new();  // resolved employee display names
}

public sealed class QueryUnderstanding
{
    public Intent Intent { get; set; }
    public QuerySlots Slots { get; set; } = new();
}

public sealed class DomainDictionaries
{
    // Map any synonym -> canonical
    public Dictionary<string,string> Departments { get; } = new(StringComparer.OrdinalIgnoreCase);
    public Dictionary<string,string> Roles { get; } = new(StringComparer.OrdinalIgnoreCase);
    public Dictionary<string,string> Locations { get; } = new(StringComparer.OrdinalIgnoreCase);

    // Optional: label -> list of canonical roles (hierarchy buckets like "managers")
    public Dictionary<string,List<string>> RoleBuckets { get; } = new(StringComparer.OrdinalIgnoreCase);

    // Optional: list of known employee names for fuzzy matching
    public List<string> EmployeeNames { get; } = new();
}

In [3]:
var domain = new DomainDictionaries();
// Canonical names as consts
const string DeptEngineering = "Engineering";
const string DeptHR = "Human Resources";
const string DeptSales = "Sales";
const string DeptSupport = "Support";

const string RoleManager = "Manager";
const string RoleSupervisor = "Supervisor";
const string RoleTeamLead = "Team Lead";
const string RoleSoftwareEngineer = "Software Engineer";
const string RoleDirector = "Director";

const string LocSaltLakeCity = "Salt Lake City";
const string LocRemote = "Remote";
const string LocHQ = "Headquarters";

// Departments (synonym -> canonical)
foreach (var pair in new (string from, string to)[] {
    ("engineering", DeptEngineering),
    ("eng", DeptEngineering),
    ("dev", DeptEngineering),
    ("development", DeptEngineering),
    ("hr", DeptHR),
    ("human resources", DeptHR),
    ("sales", DeptSales),
    ("revenue", DeptSales),
    ("support", DeptSupport),
}) domain.Departments[pair.from] = pair.to;

// Roles (synonym -> canonical)
foreach (var pair in new (string from, string to)[] {
    ("manager", RoleManager),
    ("managers", RoleManager),
    ("supervisor", RoleSupervisor),
    ("supervisors", RoleSupervisor),
    ("super", RoleSupervisor), // common short form
    ("supervisor", RoleSupervisor),
    ("supervisors", RoleSupervisor),
    ("sup", RoleSupervisor),
    ("team lead", RoleTeamLead),
    ("lead", RoleTeamLead),
    ("leads", RoleTeamLead),
    ("developer", RoleSoftwareEngineer),
    ("dev", RoleSoftwareEngineer),
    ("engineer", RoleSoftwareEngineer),
    ("software engineer", RoleSoftwareEngineer),
    ("software developer", RoleSoftwareEngineer),
    ("software engineer", RoleSoftwareEngineer),
    ("software dev", RoleSoftwareEngineer),
    ("engineer", RoleSoftwareEngineer),
    ("director", RoleDirector),
    ("dir", RoleDirector),
    ("directors", RoleDirector),
    ("head of", RoleDirector), // common phrasing
    ("head of department", RoleDirector),
    ("head of engineering", RoleDirector),
    ("head of hr", RoleDirector),
    ("head of sales", RoleDirector),
    ("head of support", RoleDirector),
    ("head of development", RoleDirector),
}) domain.Roles[pair.from] = pair.to;

// Locations (synonym -> canonical)
foreach (var pair in new (string from, string to)[] {
    ("slc", LocSaltLakeCity),
    ("salt lake", LocSaltLakeCity),
    ("salt lake city", LocSaltLakeCity),
    ("remote", LocRemote),
    ("hq", LocHQ),
}) domain.Locations[pair.from] = pair.to;

// Role buckets (category -> list of canonical roles)
domain.RoleBuckets["managers"] = new List<string> { RoleManager, RoleSupervisor, RoleTeamLead, RoleDirector };

// Known names (optional, for fuzzy)
domain.EmployeeNames.AddRange(new [] {
    "Rick Sanchez","Summer Smith","Morty Smith","Beth Smith","Jerry Smith",
    "Alice Johnson","Bob Lee","Carol Danvers","Tony Stark","Bruce Wayne"
});

In [4]:
static string Normalize(string s) =>
    Regex.Replace(s, @"\s+", " ").Trim();

static bool ContainsAny(string text, params string[] terms) =>
    terms.Any(t => text.IndexOf(t, StringComparison.OrdinalIgnoreCase) >= 0);

// map synonyms using dictionary; returns canonical if matched
static string? MapCanonical(string text, Dictionary<string,string> dict)
{
    // exact/single-token match first
    if (dict.TryGetValue(text, out var canonical)) return canonical;

    // contains-phrase match
    foreach (var kvp in dict)
    {
        if (text.Contains(kvp.Key, StringComparison.OrdinalIgnoreCase))
            return kvp.Value;
    }
    return null;
}

In [5]:
var culture = Culture.English;
var dateModel = new DateTimeRecognizer(culture, options: DateTimeOptions.None, lazyInitialization: true)
    .GetDateTimeModel();
var numberModel = new NumberRecognizer(culture).GetNumberModel();

In [6]:
QuerySlots ExtractSlots(string text, DomainDictionaries domain)
{
    text = Normalize(text);
    var slots = new QuerySlots();

    // Field + operator from phrasing
    if (ContainsAny(text, "hire date","hired","start date","joined"))
    {
        slots.Field = "OriginalHireDate";
        if (ContainsAny(text, "before","earlier than","prior to")) slots.Operator = "before";
        else if (ContainsAny(text, "after","later than","since")) slots.Operator = "after";
        else if (ContainsAny(text, "between","from","to","range")) slots.Operator = "between";
    }

    // Department / Role / Location via domain dictionaries
    slots.Department = MapCanonical(text, domain.Departments);
    slots.Role = MapCanonical(text, domain.Roles);
    slots.Location = MapCanonical(text, domain.Locations);

    // Date parsing (supports "before 2024", "last year", ranges, etc.)
    var dateResults = dateModel.Parse(text);
    // pick first resolution; improve as needed
    foreach (var r in dateResults)
    {
        if (!r.Resolution.TryGetValue("values", out var valuesObj)) continue;
        if (valuesObj is not List<Dictionary<string, string>> values) continue;

        foreach (var v in values)
        {
            if (!v.TryGetValue("type", out var type)) continue;
            if (type == "date" && v.TryGetValue("value", out var val))
            {
                if (DateTime.TryParse(val, out var d))
                {
                    if (slots.Operator == "before") { slots.DateEnd = d; }
                    else if (slots.Operator == "after") { slots.DateStart = d; }
                    else if (slots.DateStart is null) { slots.DateStart = d; }
                    else if (slots.DateEnd is null) { slots.DateEnd = d; }
                }
            }
            else if (type == "daterange")
            {
                if (v.TryGetValue("start", out var s) && DateTime.TryParse(s, out var ds))
                    slots.DateStart = ds;
                if (v.TryGetValue("end", out var e) && DateTime.TryParse(e, out var de))
                    slots.DateEnd = de;
                slots.Operator ??= "between";
            }
        }
    }

    // Names (fuzzy to known employees). Keep top hits that meet a threshold.
    var tokens = text.Split(new[]{' ', ',', ';'}, StringSplitOptions.RemoveEmptyEntries);
    var uniqueTokens = tokens.Select(t => t.Trim(new[]{'.','?','!'})).Where(t => t.Length > 1).Distinct(StringComparer.OrdinalIgnoreCase);
    foreach (var token in uniqueTokens)
    {
        var best = Process.ExtractOne(token, domain.EmployeeNames);
        if (best != null && best.Score >= 88) // threshold
        {
            if (!slots.Names.Contains(best.Value))
                slots.Names.Add(best.Value);
        }
    }

    return slots;
}

In [7]:
// If you have a trained ML.NET model, load it here. Otherwise, use simple rules.
// string? modelPath = null; // e.g., @"./models/intent_model.zip"
string modelPath = @"/Users/maneki-neko/learning/NLP/intent_model.zip";

PredictionEngine<QueryRecord, IntentPrediction>? engine = null;
MLContext ml = new MLContext(seed: 123);

if (!string.IsNullOrWhiteSpace(modelPath) && System.IO.File.Exists(modelPath))
{
    using var fs = System.IO.File.OpenRead(modelPath);
    var trained = ml.Model.Load(fs, out var _);
    engine = ml.Model.CreatePredictionEngine<QueryRecord, IntentPrediction>(trained);
}

// schema for model - must match the trained model
public sealed class QueryRecord 
{ 
    public string Text { get; set; } = ""; 
    public string Label { get; set; } = ""; 
}
public sealed class IntentPrediction 
{ 
    public string PredictedLabel { get; set; } = ""; 
    public float[] Score { get; set; } = Array.Empty<float>();
}

Intent PredictIntent(string text)
{
    if (engine is not null)
    {
        var pred = engine.Predict(new QueryRecord { Text = text });
        if (Enum.TryParse<Intent>(pred.PredictedLabel, out var parsed))
            return parsed;
    }

    // Fallback rules (fast & predictable)
    if (ContainsAny(text, "email","e-mail","mail")) return Intent.GET_CONTACT_EMAIL;
    if (ContainsAny(text, "phone","cell","mobile","call")) return Intent.GET_CONTACT_PHONE;
    if (ContainsAny(text, "address","location","where is")) return Intent.GET_CONTACT_ADDRESS;
    if (ContainsAny(text, "hire date","hired","start date","joined","before","after","between"))
        return Intent.FILTER_BY_HIRE_DATE;
    if (ContainsAny(text, "department","engineering","hr","sales","support"))
        return Intent.FILTER_BY_DEPARTMENT;
    if (ContainsAny(text, "office","remote","salt lake","hq","slc"))
        return Intent.FILTER_BY_LOCATION;
    if (ContainsAny(text, "birthday","birth date","turns","age"))
        return Intent.FILTER_BY_BIRTHDAY;

    return Intent.UNKNOWN;
}

In [8]:
QueryUnderstanding Understand(string text, DomainDictionaries domain)
{
    var intent = PredictIntent(text);
    var slots = ExtractSlots(text, domain);
    return new QueryUnderstanding { Intent = intent, Slots = slots };
}

In [12]:
var queries = new []
{
    "Show me managers in engineering hired before 2024",
    "Emails for rick, summer and morty",
    "All employees between 2020 and 2023 in HR",
    "Who is remote in SLC?",
    "Phone for Carol Danvers",
    "Directors hired after last year in sales"
};

Console.Clear();
foreach (var q in queries)
{
    var u = Understand(q, domain);
    Console.WriteLine($"Q: {q}");
    Console.WriteLine($"  Intent: {u.Intent}");
    var s = u.Slots;
    Console.WriteLine($"  Slots: field={s.Field}, op={s.Operator}, start={s.DateStart}, end={s.DateEnd}, dept={s.Department}, role={s.Role}, loc={s.Location}");
    if (s.Names.Count > 0) Console.WriteLine("  Names: " + string.Join(", ", s.Names));
    Console.WriteLine();
}

Q: Show me managers in engineering hired before 2024
  Intent: FILTER_BY_DEPARTMENT
  Slots: field=OriginalHireDate, op=before, start=, end=1/1/2024 12:00:00 AM, dept=Engineering, role=Manager, loc=
  Names: Summer Smith

Q: Emails for rick, summer and morty
  Intent: GET_CONTACT_EMAIL
  Slots: field=, op=between, start=, end=, dept=, role=, loc=
  Names: Rick Sanchez, Summer Smith, Morty Smith

Q: All employees between 2020 and 2023 in HR
  Intent: FILTER_BY_HIRE_DATE
  Slots: field=, op=between, start=1/1/2020 12:00:00 AM, end=1/1/2023 12:00:00 AM, dept=Human Resources, role=, loc=

Q: Who is remote in SLC?
  Intent: FILTER_BY_LOCATION
  Slots: field=, op=, start=, end=, dept=, role=, loc=Salt Lake City

Q: Phone for Carol Danvers
  Intent: GET_CONTACT_PHONE
  Slots: field=, op=, start=, end=, dept=, role=, loc=
  Names: Carol Danvers

Q: Directors hired after last year in sales
  Intent: FILTER_BY_HIRE_DATE
  Slots: field=OriginalHireDate, op=after, start=1/1/2025 12:00:00 AM, end=,